In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense,Input,Bidirectional,LSTM,GRU,SimpleRNN,Dropout,LayerNormalization , Concatenate,Conv2D,MaxPooling2D,Layer,BatchNormalization,GlobalMaxPooling1D,Conv1D,Activation, Dot
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.utils import class_weight
import librosa
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
import pickle
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tensorflow.keras.models import clone_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import plot_model
import pickle

2024-07-04 09:51:47.873369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 09:51:47.873474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 09:51:47.969767: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
file_path = '/content/drive/MyDrive/padded_mfccs_combined_augmented.pickle'

# Open the file in binary read mode
with open(file_path, 'rb') as f:
    # Load the data from the file
    mfccs = pickle.load(f)

    
    
file_path = '/content/drive/MyDrive/labels_combined_augmented.pickle'

# Open the file in binary read mode
with open(file_path, 'rb') as f:
    # Load the data from the file
    labels = pickle.load(f)

In [4]:
early_stopping = EarlyStopping(
    monitor='val_loss',      # Metric to monitor
    patience=5,              # Number of epochs with no improvement after which training will be stopped
    verbose=1,               # Verbosity mode
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',     # Metric to monitor
    factor=0.5,             # Factor by which the learning rate will be reduced
    patience=3,             # Number of epochs with no improvement after which learning rate will be reduced
    verbose=1,              # Verbosity mode
    min_lr=1e-7             # Lower bound on the learning rate
)


In [5]:
X_mfccs = np.array(mfccs)
y = np.array(labels)

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)


In [7]:
# Define the input shape
input_shape = (740, 13, 1)  # Adding a channel dimension for Conv2D

# Input layer
input_layer = Input(shape=input_shape)

# BiLSTM branch
bilstm_input_model1 = tf.keras.layers.Reshape((740, 13))(input_layer)  # Reshaping for LSTM input
bilstm1_model1 = Bidirectional(LSTM(64,
                            return_sequences=True
                            ))(bilstm_input_model1)

# Add second Bidirectional LSTM layer with regularizations
bilstm2_model1 = Bidirectional(LSTM(64
                            ))(bilstm1_model1)


# Conv2D branch
conv2d = Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(input_layer)
conv2d = BatchNormalization()(conv2d)

conv2d = MaxPooling2D(pool_size=(2, 2))(conv2d)
# dropout1 = Dropout(0.5)(conv2d)
conv2d = Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same')(conv2d)
conv2d = BatchNormalization()(conv2d)
conv2d = MaxPooling2D(pool_size=(2, 2))(conv2d)
# dropout2 = Dropout(0.5)(conv2d)
conv2d = Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same')(conv2d)
conv2d = BatchNormalization()(conv2d)
conv2d = MaxPooling2D(pool_size=(2, 2))(conv2d)
# dropout3 = Dropout(0.5)(conv2d)
conv_flat = Flatten()(conv2d)
dense1 = Dense(512,activation = 'relu')(conv_flat)
dropout4 = Dropout(0.2)(dense1)
dense2 = Dense(128,activation ='relu')(dropout4)
dropout5 = Dropout(0.2)(dense2)
# Concatenate BiLSTM and Conv2D branches
concatenated = Concatenate()([bilstm2_model1, dropout5])

dense = Dense(64,activation='relu')(concatenated)
output_layer = Dense(5, activation='softmax')(dense)
model_speech = Model(inputs=input_layer,outputs=output_layer)

model_speech.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the summary of the combined model
model_speech.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 740, 13,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 740, 13,   │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 740, 13,   │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 370, 6,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 370, 6,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 370, 6,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 185, 3,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 185, 3,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 185, 3,    │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 92, 1,     │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 11776)     │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │  6,029,824 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 740, 13)   │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 740, 128)  │     39,936 │ reshape[0][0]     │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     65,664 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 128)       │     98,816 │ bidirectional[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ bidirectional_1[

 Total params: 6,344,581 (24.20 MB)

 Trainable params: 6,344,133 (24.20 MB)

 Non-trainable params: 448 (1.75 KB)

In [8]:
val_accuracies = []
train_accuracies_per_epoch = []
val_accuracies_per_epoch = []
train_losses_per_epoch = []
val_losses_per_epoch = []

for train_index, val_index in kf.split(X_mfccs):
    X_mfccs_train, X_mfccs_val = X_mfccs[train_index], X_mfccs[val_index]
    y_train, y_val = y[train_index], y[val_index]

    history = model_speech.fit(
        X_mfccs_train,                 # Input data
        y_train,                       # Labels
        batch_size=32,                 # Batch size
        epochs=30,                     # Number of epochs
        validation_data=(X_mfccs_val, y_val),
        callbacks=[reduce_lr,early_stopping]
    )
    val_accuracy = history.history['val_accuracy'][-1]
    val_accuracies.append(val_accuracy)
    print(f"Fold validation accuracy: {val_accuracy}")

    # Store the accuracies
    train_accuracies_per_epoch.append(history.history['accuracy'])
    val_accuracies_per_epoch.append(history.history['val_accuracy'])
    train_losses_per_epoch.append(history.history['loss'])
    val_losses_per_epoch.append(history.history['val_loss'])


average_val_accuracy = np.mean(val_accuracies)
print("Average Validation Accuracy:", average_val_accuracy)




Epoch 1/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 30s 131ms/step - accuracy: 0.2891 - loss: 2.1737 - val_accuracy: 0.4117 - val_loss: 1.3588 - learning_rate: 0.0010
Epoch 2/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 16s 111ms/step - accuracy: 0.4120 - loss: 1.3553 - val_accuracy: 0.4064 - val_loss: 1.3347 - learning_rate: 0.0010
Epoch 3/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 16s 112ms/step - accuracy: 0.4081 - loss: 1.3451 - val_accuracy: 0.4064 - val_loss: 1.3792 - learning_rate: 0.0010
Epoch 4/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 16s 112ms/step - accuracy: 0.4460 - loss: 1.3153 - val_accuracy: 0.4534 - val_loss: 1.2573 - learning_rate: 0.0010
Epoch 5/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 16s 112ms/step - accuracy: 0.4533 - loss: 1.2383 - val_accuracy: 0.4712 - val_loss: 1.2382 - learning_rate: 0.0010
Epoch 6/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 16s 112ms/step - accuracy: 0.4584 - loss: 1.2489 - val_accuracy: 0.4614 - val_loss: 1.2296 - learning_rate: 0.0010
Epoch 7/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 16s 113ms/step - accuracy: 0.4

In [10]:
import numpy as np

# Assuming you have a test dataset X_test and true labels y_true
# Make predictions on the test dataset
predictions = model_speech.predict(X_mfccs_val)

# Convert predicted probabilities to class labels
predicted_classes = np.argmax(predictions, axis=-1)

# Convert true labels to class labels
true_classes = np.argmax(y_val, axis=-1)

# Check the shapes of predicted_classes and true_classes
print("Shapes of predicted_classes and true_classes:")
print(predicted_classes.shape)
print(true_classes.shape)

# Calculate overall accuracy
overall_accuracy = np.mean(predicted_classes == true_classes)
print(f"\nOverall Accuracy: {overall_accuracy}")

# If necessary, calculate class-wise accuracy
if len(np.unique(true_classes)) > 1:
    # Calculate class-wise accuracy
    class_accuracy = {}
    for class_label in range(5):  # Assuming num_classes is the number of classes (5 in your case)
        # Indices where true labels match the current class label
        class_indices = np.where(true_classes == class_label)[0]

        # Indices where predicted labels match the current class label
        correct_indices = np.where(predicted_classes[class_indices] == class_label)[0]

        # Calculate accuracy for the current class
        if len(class_indices) > 0:
            accuracy = len(correct_indices) / len(class_indices)
        else:
            accuracy = 0.0

        # Store class-wise accuracy
        class_accuracy[class_label] = accuracy

    # Display or store class-wise accuracy
    print("\nClass-wise Accuracy:")
    for class_label, accuracy in class_accuracy.items():
        print(f"Class {class_label}: Accuracy {accuracy}")
else:
    print("\nClass-wise accuracy calculation is not applicable because there is only one class present.")


36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step
Shapes of predicted_classes and true_classes:
(1126,)
(1126,)

Overall Accuracy: 0.7184724689165186

Class-wise Accuracy:
Class 0: Accuracy 0.7873303167420814
Class 1: Accuracy 0.7102803738317757
Class 2: Accuracy 0.7280334728033473
Class 3: Accuracy 0.5855855855855856
Class 4: Accuracy 0.7782608695652173


In [11]:
from sklearn.metrics import classification_report

# Calculate classification report
report = classification_report(true_classes, predicted_classes, digits=4)

# Print the classification report
print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0     0.8131    0.7873    0.8000       221
           1     0.7600    0.7103    0.7343       214
           2     0.7699    0.7280    0.7484       239
           3     0.5394    0.5856    0.5616       222
           4     0.7306    0.7783    0.7537       230

    accuracy                         0.7185      1126
   macro avg     0.7226    0.7179    0.7196      1126
weighted avg     0.7230    0.7185    0.7201      1126

